In [1]:
import numpy as np
import pandas as pd
import re

from bs4 import BeautifulSoup
import requests

In [24]:
base_url = 'https://boardgamegeek.com/'
start_url = 'https://boardgamegeek.com/browse/boardgame/page/1'

In [5]:
test = requests.get(start_url).text

In [6]:
soup = BeautifulSoup(test,'lxml')

In [17]:
soup

<!DOCTYPE html>
<html lang="en-US" ng-app="GeekApp" ng-cloak="">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" id="vp" name="viewport"/>
<script>
			window.addEventListener( 'DOMContentLoaded',  function() {
				if (window.innerWidth < 960) {
					var mvp = document.getElementById('vp');
					mvp.setAttribute('content','width=960');
				}
			});
		</script>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Browse Board Games | BoardGameGeek</title>
<link href="https://cf.geekdo-static.com/icons/touch-icon180.png" rel="apple-touch-icon"/>
<link href="https://cf.geekdo-static.com/icons/favicon2.ico" rel="shortcut icon" type="image/ico"/>
<link href="https://cf.geekdo-static.com/icons/favicon2.ico" rel="icon" type="image/ico"/>
<link href="/game-opensearch.xml" rel="search" title="BGG Game Search" type="application/opensearchdescription+xml"/>
<meta content="BGG" name="app

In [25]:
game_l = soup.find_all('td',{'class':'collection_objectname'})

In [29]:
game_l[0].select('a')[0]['href']

'/boardgame/174430/gloomhaven'

In [ ]:
page_n = 1
games = []
start_url = 'https://boardgamegeek.com/browse/boardgame/page/'
while True:
    game_names = BeautifulSoup(requests.get(start_url+str(page_n)).text,'lxml')
    game_list = soup.find_all('td',{'class':'collection_objectname'})
    if not game_list:
        print('No more results: Exiting...') #check to see if results exits, other wise break
        break
    else:
        print('Getting page:{}'.format(page_n))
        
        
        for game in game_list:
            game_page = requests.get(base_url+game.select('a')[0]['href']+'/stats').text
    

In [ ]:
page_n = 1
books = []
while True:
    html_titles = requests.get(start_url+str(page_n)).text #get page 'i'
    titles = BeautifulSoup(html_titles, 'lxml') #soupify
    prod_list = titles.find_all('div',{"class":"list_item"}) #get list of titles
    if not prod_list:
        print('No more results: Exiting...') #check to see if results exits, other wise break
        break
    else:
        print('Getting page:{}'.format(page_n))
        
        page_links = [prod_list[i].select('a')[1]['href'] for i in range(len(prod_list))] #list of links
        for link in page_links: #iterate through links)
            data = []
            request = requests.get(base_url + link).text
            soup = BeautifulSoup(request, 'lxml')            
            data.append(link.split('/')[2]) #append job id
            data.append(soup.find_all('h2',{'class':'title'})[0].text) #append title
            data.append(soup.find_all('div',{'class':'product-description'})[0].text) #append description
            spec = [i.text[:-1] for i in soup.find_all('dt')] #generate list of specs
            res = [i.text for i in soup.find_all('dd')] #generate spec details
            d = {i:j for i,j in zip(spec,res)}
            for i in base_specs: #iterate through base_specs; if present add to data, else append 'Na'
                if i in d.keys():
                    data.append(d[i])
                else:
                    data.append('Na')
            
            books.append(data) #append book data to list of books       
    page_n += 1 #go to next page